We compute adjoint kernels in python directly. When computed in AxiSEM3D, there is 2 orders magnitude error. When computing a single slice in python, the amplitudes seem correct. We thus want to compute the 3D kernels to make sure.

In [1]:
import numpy as np
from netCDF4 import Dataset
from mpi4py import MPI

In [2]:
### INPUTS

INPUT_FILE_FWD = '/home/alex/Desktop/phd/axisem3d_alex/AxiSEM3D/build/output/wavefields/wavefield_db_fwd.nc4'
INPUT_FILE_ADJ = '/home/alex/Desktop/phd/axisem3d_alex/AxiSEM3D/build/output/wavefields/wavefield_db.nc4'

START_TSTEP = 0
END_TSTEP = 100
INT_TSTEP = 1

In [3]:
### start up MPI 

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print(rank)

0


In [ ]:
### master loads files

if (rank == 0): 
    dset_fwd = Dataset(INPUT_FILE_FWD)
    dset_ker = Dataset(INPUT_FILE_ADJ)

    wavefield = dset_fwd.variables["displacement_wavefield"][END_TSTEP:START_TSTEP:-INT_TSTEP] #domain decomposed wavefield.
    kernels = dset_ker.variables["displacement_wavefield"][START_TSTEP:END_TSTEP:INT_TSTEP]
    sem_mesh = dset_fwd.variables["sem_mesh"][:] #for each point contains global point tag
    s_temp = dset_fwd.variables["mesh_S"][:]
    z_temp = dset_fwd.variables["mesh_Z"][:]
    nu = dset_fwd.variables['Nus'][:]
    nuKer = dset_ker.variables['Nus'][:]
    vp_temp = dset_fwd.variables['vp1D'][:]
    vp = np.expand_dims(vp_temp, axis = 0) # allows to treat it as a wvf
    rho_temp = dset_fwd.variables['rho1D'][:]
    rho = np.expand_dims(rho_temp, axis = 0) # allows to treat it as a wvf
    int_factor = dset_fwd.variables['integral_factor'][:]
    npoints = np.max(sem_mesh) + 1
    nelem = len(nu)
    s = np.zeros(npoints, dtype = np.float32)
    z = np.zeros(npoints, dtype = np.float32)
    tim = dset_fwd.variables["time"][:]
    dt = tim[1] - tim[0]